# INCEPTION V3
### This is our Transfer Learning final model which allowed us to reach 0,937 of accuracy on the test set

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os

cwd = os.getcwd()
dataset_dir = '../input/maskdataset/MaskDataset'

# Training Directory
train_data_dir = os.path.join(dataset_dir, 'training')

# Validation Directory
valid_data_dir = os.path.join(dataset_dir, 'validation')

### Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

# Create training ImageDataGenerator object
train_data_gen = ImageDataGenerator( rotation_range=transformation_ratio,
                                     shear_range=transformation_ratio,
                                     zoom_range=transformation_ratio,
                                     cval=transformation_ratio,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     rescale=1./255 )

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

### Dataset Generators

In [ ]:
import tensorflow as tf
import numpy as np

# Fix seed for reproducible results
seed = 9
np.random.seed(seed=seed)
tf.random.set_seed(seed)

# Number of classes
num_classes = 3  

# Img shape
img_h, img_w = 408, 612  

# Batch size
batch_size = 8 

# Training
train_gen = train_data_gen.flow_from_directory( train_data_dir,
                                                target_size=(img_h, img_w),
                                                batch_size=batch_size,
                                                class_mode='categorical')

# Validation
valid_gen = valid_data_gen.flow_from_directory(valid_data_dir,
                                               target_size=(img_h, img_w),
                                               batch_size=batch_size,
                                               class_mode='categorical')
# Test
test_data_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_data_dir,
                                             target_size=(img_h, img_w),
                                             batch_size=batch_size, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=seed)


# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
train_dataset = train_dataset.repeat()



# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
valid_dataset = valid_dataset.repeat()



# Test
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
test_dataset = valid_dataset.repeat()

### Model Creation

In [ ]:
import keras
from keras import backend as K
from keras.layers import *

#Image data is represented in a three-dimensional array
input_shape = (img_h, img_w, 3) #rows, columns, channels

#Load InceptionV3 model    
base_model = tf.keras.applications.InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)

# blocking top model   
x = base_model.output
x = GlobalAveragePooling2D()(x)                   # GAP gives us the vector with averages
x = Dense(512, activation='relu', name='fc1')(x)  # only one fully connected layer
x = Dropout(0.5)(x)                               # to prevent overfitting

# last layer
predictions = Dense(num_classes, activation='softmax', name='predictions')(x)

# add your top layer block to your base model
model = keras.models.Model(base_model.input, predictions)

# bottleneck featuring 
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the based model that is already pre-trained.

for layer in base_model.layers:
    layer.trainable = False

### Visualise our model layers

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

### Model Compiling


In [ ]:
from keras.optimizers import *
from keras.applications import *

lr = 1e-3  # sgd learning rate

model.compile(optimizer='nadam',
              loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
              metrics=['accuracy'])

### Fine Tuning
#### freeze_until points to the layer in our model we want to train will be re-trained based on the new data.

In [ ]:
freeze_until = 126  # value is based on based model selected.

for layer in model.layers[:freeze_until]:
    layer.trainable = False
for layer in model.layers[freeze_until:]:
    layer.trainable = True

model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])   

### Load the extension and start TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

### Callback and Early Stopping


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Configure the TensorBoard callback and fit your model

tb_callback = tf.keras.callbacks.TensorBoard("logs")

callbacks = []

callbacks.append(tb_callback)

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
    callbacks.append(es_callback)

### Fit the Model

In [ ]:
model.fit(x=train_dataset,
          epochs=100,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=callbacks)